In [1]:
import string
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from numpy import array
from random import randint
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
# load doc into memory
def load_doc( filename ):
    
    # open the file as read only
    file = open( filename, 'r' )
    # read all text
    text = file.read()
    # close the file
    file.close()
    
    return text

# load document
#in_filename = "../texts/alice-in-wonderland.txt"
in_filename = "../texts/dr-zeuss-compilation.txt"
doc = load_doc( in_filename )
print( doc[ :200 ] )

The Cat in the Hat

By Dr. Seuss

The sun did not shine.
It was too wet to play.
So we sat in the house
All that cold, cold, wet day.

I sat there with Sally.
We sat there, we two.
And I said, "How I 


In [3]:
# turn a doc into clean tokens
def clean_doc( doc, to_lower=True ):
    
    # replace '--' with a space ' '
    doc = doc.replace( '--', ' ' )
    
    # split into tokens by white space
    tokens = doc.split()
    
    # remove punctuation from each token
    table = str.maketrans( '', '', string.punctuation )
    tokens = [ w.translate( table ) for w in tokens ]
    
    # remove remaining tokens that are not alphabetic
    if to_lower:
        tokens = [ word for word in tokens if word.isalpha() ]
    
    # make lower case
    tokens = [ word.lower() for word in tokens ] 
    
    return tokens

In [4]:
# clean document
tokens = clean_doc( doc )
print( tokens[ :200 ] )
print( 'Total Tokens: %d' % len( tokens ) )
print( 'Unique Tokens: %d' % len( set( tokens ) ) )

['the', 'cat', 'in', 'the', 'hat', 'by', 'dr', 'seuss', 'the', 'sun', 'did', 'not', 'shine', 'it', 'was', 'too', 'wet', 'to', 'play', 'so', 'we', 'sat', 'in', 'the', 'house', 'all', 'that', 'cold', 'cold', 'wet', 'day', 'i', 'sat', 'there', 'with', 'sally', 'we', 'sat', 'there', 'we', 'two', 'and', 'i', 'said', 'how', 'i', 'wish', 'we', 'had', 'something', 'to', 'do', 'too', 'wet', 'to', 'go', 'out', 'and', 'too', 'cold', 'to', 'play', 'ball', 'so', 'we', 'sat', 'in', 'the', 'house', 'we', 'did', 'nothing', 'at', 'all', 'so', 'all', 'we', 'could', 'do', 'was', 'to', 'sit', 'sit', 'sit', 'sit', 'and', 'we', 'did', 'not', 'like', 'it', 'not', 'one', 'little', 'bit', 'bump', 'and', 'then', 'something', 'went', 'bump', 'how', 'that', 'bump', 'made', 'us', 'jump', 'we', 'looked', 'then', 'we', 'saw', 'him', 'step', 'in', 'on', 'the', 'mat', 'we', 'looked', 'and', 'we', 'saw', 'him', 'the', 'cat', 'in', 'the', 'hat', 'and', 'he', 'said', 'to', 'us', 'why', 'do', 'you', 'sit', 'there', 'like'

In [5]:
# organize into sequences of tokens
sequence_len = 50 + 1
sequences = list()

for i in range( sequence_len, len( tokens ) ):
    
    # select sequence of tokens
    seq = tokens[ i - sequence_len:i ]
    
    # convert into a line
    line = ' '.join( seq )
    
    # store
    sequences.append( line )
    
print( 'Total Sequences: %d' % len( sequences ) )


Total Sequences: 6239


In [6]:
# save tokens to file, one dialog per line
def save_doc( lines, filename ):
    
    data = '\n'.join( lines )
    file = open( filename, 'w' )
    file.write( data )
    file.close()

In [7]:
# save sequences to file
out_filename = "../texts/dr-zeuss-compilation-sequences.txt"
save_doc( sequences, out_filename )

In [8]:
in_filename = "../texts/dr-zeuss-compilation-sequences.txt"
doc = load_doc( in_filename )
lines = doc.split( '\n' )
lines[ 0:10 ]

['the cat in the hat by dr seuss the sun did not shine it was too wet to play so we sat in the house all that cold cold wet day i sat there with sally we sat there we two and i said how i wish we had something to',
 'cat in the hat by dr seuss the sun did not shine it was too wet to play so we sat in the house all that cold cold wet day i sat there with sally we sat there we two and i said how i wish we had something to do',
 'in the hat by dr seuss the sun did not shine it was too wet to play so we sat in the house all that cold cold wet day i sat there with sally we sat there we two and i said how i wish we had something to do too',
 'the hat by dr seuss the sun did not shine it was too wet to play so we sat in the house all that cold cold wet day i sat there with sally we sat there we two and i said how i wish we had something to do too wet',
 'hat by dr seuss the sun did not shine it was too wet to play so we sat in the house all that cold cold wet day i sat there with sally we sat

## Convert Words to Index Values

In [9]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts( lines )
sequences = tokenizer.texts_to_sequences( lines )

In [10]:
len( sequences[ 0 ] ) == sequence_len

True

In [11]:
print( len( tokenizer.word_index ) )
print( type( tokenizer.word_index ) )
print( tokenizer.word_index[ "cat" ] )

855
<class 'dict'>
37


In [12]:
# vocabulary size
vocab_size = len( tokenizer.word_index ) + 1
vocab_size

856

In [13]:
# separate into input and output: for now it's 50 words input and 1 word output
sequences = array( sequences )
X = sequences[ :,:-1 ] # all rows, from word 0 up to, but not including, the last word
y = sequences[ :,-1 ]  # all rows, last word only
y = to_categorical( y, num_classes=vocab_size )
seq_length = X.shape[ 1 ]

In [14]:
seq_length

50

## Define Model

In [15]:
import keras
print( keras.__version__ )

import tensorflow as tf
print( tf.__version__ )

2.1.3
1.4.1


In [17]:
# define model
model = Sequential()
# original embedding size was 50, common values are 50, 100, and 300: dimension used to represent each word.
# we could use pre-calc'd embeddings too: https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/
model.add( Embedding( vocab_size, 300, input_length=seq_length ) )
model.add( LSTM( seq_length * 2, return_sequences=True ) )
model.add( LSTM( seq_length * 2 ) )
model.add( Dense( seq_length * 2, activation='relu' ) )

# fixed TypeError below, downgraded keras from 2.1.5 to 2.1.3: https://github.com/keras-team/keras/issues/9621
# TypeError: softmax() got an unexpected keyword argument 'axis'
model.add( Dense( vocab_size, activation='softmax' ) )

print( model.summary() )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 300)           256800    
_________________________________________________________________
lstm_3 (LSTM)                (None, 50, 100)           160400    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 856)               86456     
Total params: 594,156
Trainable params: 594,156
Non-trainable params: 0
_________________________________________________________________
None


## Fit the Model

In [18]:
# calc batch size
print( len( sequences ) / 128 )
print( len( sequences ) / 1028 )
# Was:
# batch_size = 128
batch_size = 1028
# batch_size = 32


48.7421875
6.069066147859922


In [19]:
# compile model
model.compile( loss='categorical_crossentropy', optimizer='adam', metrics=[ 'accuracy' ] )
# fit model
model.fit( X, y, batch_size=batch_size, epochs=300 )

Epoch 1/300
6239/6239 [==============================] - 2s 250us/step - loss: 6.7474 - acc: 0.0237
Epoch 2/300
6239/6239 [==============================] - 1s 120us/step - loss: 6.6137 - acc: 0.0321
Epoch 3/300
6239/6239 [==============================] - 1s 120us/step - loss: 5.9919 - acc: 0.0321
Epoch 4/300
6239/6239 [==============================] - 1s 120us/step - loss: 5.7363 - acc: 0.0365
Epoch 5/300
6239/6239 [==============================] - 1s 120us/step - loss: 5.6337 - acc: 0.0338
Epoch 6/300
6239/6239 [==============================] - 1s 120us/step - loss: 5.6325 - acc: 0.0327
Epoch 7/300
6239/6239 [==============================] - 1s 120us/step - loss: 5.6164 - acc: 0.0389
Epoch 8/300
6239/6239 [==============================] - 1s 120us/step - loss: 5.6166 - acc: 0.0385
Epoch 9/300
6239/6239 [==============================] - 1s 120us/step - loss: 5.6108 - acc: 0.0385
Epoch 10/300
6239/6239 [==============================] - 1s 120us/step - loss: 5.6105 - acc: 0.0341

6239/6239 [==============================] - 1s 121us/step - loss: 4.1391 - acc: 0.1380
Epoch 83/300
6239/6239 [==============================] - 1s 121us/step - loss: 4.1196 - acc: 0.1377
Epoch 84/300
6239/6239 [==============================] - 1s 121us/step - loss: 4.0978 - acc: 0.1420
Epoch 85/300
6239/6239 [==============================] - 1s 121us/step - loss: 4.0745 - acc: 0.1537
Epoch 86/300
6239/6239 [==============================] - 1s 122us/step - loss: 4.0620 - acc: 0.1523
Epoch 87/300
6239/6239 [==============================] - 1s 122us/step - loss: 4.0417 - acc: 0.1572
Epoch 88/300
6239/6239 [==============================] - 1s 121us/step - loss: 4.0305 - acc: 0.1492
Epoch 89/300
6239/6239 [==============================] - 1s 121us/step - loss: 4.0047 - acc: 0.1571
Epoch 90/300
6239/6239 [==============================] - 1s 121us/step - loss: 3.9895 - acc: 0.1628
Epoch 91/300
6239/6239 [==============================] - 1s 121us/step - loss: 3.9720 - acc: 0.1654
Epo

6239/6239 [==============================] - 1s 122us/step - loss: 2.7056 - acc: 0.3525
Epoch 163/300
6239/6239 [==============================] - 1s 122us/step - loss: 2.6851 - acc: 0.3486
Epoch 164/300
6239/6239 [==============================] - 1s 122us/step - loss: 2.6794 - acc: 0.3502
Epoch 165/300
6239/6239 [==============================] - 1s 122us/step - loss: 2.6606 - acc: 0.3594
Epoch 166/300
6239/6239 [==============================] - 1s 122us/step - loss: 2.6384 - acc: 0.3600
Epoch 167/300
6239/6239 [==============================] - 1s 122us/step - loss: 2.6165 - acc: 0.3659
Epoch 168/300
6239/6239 [==============================] - 1s 122us/step - loss: 2.5931 - acc: 0.3677
Epoch 169/300
6239/6239 [==============================] - 1s 122us/step - loss: 2.5827 - acc: 0.3695
Epoch 170/300
6239/6239 [==============================] - 1s 122us/step - loss: 2.5958 - acc: 0.3685
Epoch 171/300
6239/6239 [==============================] - 1s 122us/step - loss: 2.5578 - acc: 0

6239/6239 [==============================] - 1s 122us/step - loss: 1.6450 - acc: 0.5858
Epoch 243/300
6239/6239 [==============================] - 1s 122us/step - loss: 1.6424 - acc: 0.5889
Epoch 244/300
6239/6239 [==============================] - 1s 122us/step - loss: 1.6314 - acc: 0.5914
Epoch 245/300
6239/6239 [==============================] - 1s 122us/step - loss: 1.6145 - acc: 0.5996
Epoch 246/300
6239/6239 [==============================] - 1s 122us/step - loss: 1.5948 - acc: 0.6039
Epoch 247/300
6239/6239 [==============================] - 1s 122us/step - loss: 1.5835 - acc: 0.6041
Epoch 248/300
6239/6239 [==============================] - 1s 122us/step - loss: 1.5660 - acc: 0.6088
Epoch 249/300
6239/6239 [==============================] - 1s 122us/step - loss: 1.5613 - acc: 0.6121
Epoch 250/300
6239/6239 [==============================] - 1s 122us/step - loss: 1.5519 - acc: 0.6128
Epoch 251/300
6239/6239 [==============================] - 1s 122us/step - loss: 1.5233 - acc: 0

In [20]:
# save the model to file
# 00 got ~30% :-(
# 01 got 81% accuracy!
model.save( "models/dr-zeuss-compilation-01.keras" )

# save the tokenizer
dump( tokenizer, open( "tokenizers/dr-zeuss-compilation-01.pkl", 'wb' ) )

## Use The Model to Generate Text

In [21]:
seq_length = len( lines[ 0 ].split() ) - 1
seq_length

50

In [22]:
def generate_seq( model, tokenizer, seq_length, seed_text, n_words ):
    
    result = list()
    in_text = seed_text
    
    # generate a fixed number of words
    for _ in range( n_words ):
        
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences( [ in_text ] )[ 0 ] 
        
        # truncate sequences to a fixed length
        encoded = pad_sequences( [ encoded ], maxlen=seq_length, truncating='pre' ) 
        
        # predict probabilities for each word
        yhat = model.predict_classes( encoded, verbose=0 )
        
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break 
                
        # append to input
        in_text += ' ' + out_word
        
        result.append( out_word )
        
    return ' '.join( result )

In [23]:
# select a seed text
seed_text = lines[ randint( 0, len( lines ) ) ]
print( seed_text + '...\n' )
#print( len( seed_text.split( " " ) ) )

# generate new text
generated = generate_seq( model, tokenizer, seq_length, seed_text, 50 )
print( "..." + generated )

dish and look i can hop up and down on the ball but that is not all oh no that is not all look at me look at me look at me now it is fun to have fun but you have to know how i can hold up the cup...

...and the milk and the cake i will hold at me i will not oh i do not like it i will show them you oh i will not be a fish i do not will eat them here he lit good you would not like them anywhere i do


In [24]:
def generate_seq_word_by_word( model, tokenizer, seq_length, seed_text, n_words ):
    
    print( "...", end='' )
    #result = list()
    in_text = seed_text
    
    # generate a fixed number of words
    for _ in range( n_words ):
        
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences( [ in_text ] )[ 0 ] 
        
        # truncate sequences to a fixed length
        encoded = pad_sequences( [ encoded ], maxlen=seq_length, truncating='pre' ) 
        
        # predict probabilities for each word
        yhat = model.predict_classes( encoded, verbose=0 )
        
        # map predicted word index to word
        #out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                print( word, end=' ' )
                break 
                
        # append to input for next iteration
        in_text += ' ' + out_word

In [25]:
# select a seed text
seed_text = lines[ randint( 0, len( lines ) ) ]
print( seed_text + '...\n' )

# generate new text
generate_seq_word_by_word( model, tokenizer, seq_length, seed_text, 50 )

sat in the house all that cold cold wet day i sat there with sally we sat there we two and i said how i wish we had something to do too wet to go out and too cold to play ball so we sat in the house we did nothing...

...said the hat a toy ship and i cannot hear your bit is cans and some are for i like to here when this way you do to do and your hand i know this hand and this tricks and is are not one by up the yop a pet 